In [1]:
import pandas as pd
import numpy as np

# before, train 데이터 가져오기

In [4]:
before=pd.read_csv('before.csv', encoding='cp949')
train=pd.read_csv('train.csv')
train

,기준연월,방문자 수,워케이션,기타관광,레저스포츠,문화관광,쇼핑,숙박,역사관광,음식,자연관광,체험관광,쇼핑업,숙박업,식음료업,여가서비스업,여행업,운송업,전체소비
0,강원202201,11941122.0,829814.0,22405.0,58087.0,80492.0,66385.0,214092.0,22405.0,225709.0,80492.0,3319.0,15446243.0,20738225.0,53111673.0,7852979.0,230240.0,55247.0,97434607.0
1,강원202202,9689337.0,653565.0,18953.0,49017.0,58821.0,49671.0,165352.0,16993.0,192148.0,56860.0,2614.0,11926163.0,17166154.0,44641925.0,8922462.0,208316.0,47279.0,82912299.0
2,강원202203,10250173.0,1027933.0,28782.0,101765.0,79151.0,81207.0,203531.0,25698.0,318659.0,88402.0,4112.0,12280738.0,8224277.0,36738906.0,13360008.0,170724.0,31991.0,70806644.0
3,강원202204,11695493.0,693881.0,26367.0,63837.0,58286.0,47878.0,157511.0,20123.0,212328.0,64531.0,2776.0,14903256.0,11706274.0,53520764.0,21753838.0,323360.0,63098.0,102270591.0
4,강원202205,13184421.0,930414.0,28843.0,80016.0,96763.0,60477.0,214926.0,27912.0,286568.0,88389.0,3722.0,16343244.0,16380731.0,67066923.0,28034870.0,441684.0,99817.0,128367268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,충북202302,7558713.0,737216.0,106896.0,47919.0,44970.0,62663.0,67087.0,15482.0,253602.0,17693.0,5161.0,16536566.0,2902939.0,44400056.0,7450884.0,17104.0,34718.0,71342267.0
302,충북202303,10049449.0,1001610.0,142229.0,105169.0,65105.0,80129.0,84135.0,24039.0,336541.0,26042.0,6010.0,20768747.0,2902270.0,51173560.0,15319554.0,12682.0,76016.0,90252828.0
303,충북202304,9091283.0,1018608.0,148717.0,108991.0,79451.0,73340.0,99824.0,27502.0,324936.0,31577.0,8149.0,20645786.0,3211012.0,54444374.0,18263107.0,10902.0,162341.0,96737523.0
304,충북202305,9841989.0,1145340.0,160348.0,129423.0,82464.0,82464.0,115679.0,32070.0,371090.0,34360.0,6872.0,21217701.0,3267158.0,56454396.0,19760050.0,10149.0,143807.0,100853260.0


# 모델 학습

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim

# 입력 변수 x와 출력 변수 y로 나누기
x_train = train[['기타관광', '레저스포츠', '문화관광', '쇼핑', '숙박', '역사관광', '음식', '자연관광', '체험관광']]
y_train = train[['쇼핑업', '숙박업', '식음료업', '여가서비스업', '운송업', '전체소비']]

# 데이터프레임을 파이토치 텐서로 변환
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

# 모델 구축
class MultivariateModel(nn.Module):
    def __init__(self):
        super(MultivariateModel, self).__init__()
        self.fc1 = nn.Linear(9, 10)  # 첫 번째 은닉층, 9개의 입력 노드와 10개의 뉴런
        self.fc2 = nn.Linear(10, 5)  # 두 번째 은닉층, 10개의 뉴런과 5개의 뉴런
        self.fc3 = nn.Linear(5, 6)   # 출력층, 5개의 뉴런과 6개의 출력 노드

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # 첫 번째 은닉층의 활성화 함수는 ReLU
        x = torch.relu(self.fc2(x))  # 두 번째 은닉층의 활성화 함수는 ReLU
        x = self.fc3(x)  # 출력층에는 활성화 함수를 사용하지 않음
        return x

model = MultivariateModel()

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(x_train_tensor)  # 수정: x_train_tensor 사용
    loss = criterion(outputs, y_train_tensor)  # 수정: y_train_tensor 사용
    loss.backward()
    optimizer.step()

# 지니 계수 계산 함수 정의

In [ ]:
def gini_coefficient(data):
    # 데이터를 오름차순으로 정렬
    sorted_data = np.sort(data)

    # 누적 상대 빈도 계산
    cumulative_freq = np.cumsum(sorted_data) / np.sum(sorted_data)

    # 누적 상대 누적값 계산
    cumulative_lorenz = np.cumsum(cumulative_freq) / np.sum(cumulative_freq)

    # Gini 계수 계산
    area_under_curve = np.trapz(cumulative_lorenz, x=cumulative_freq)
    gini_coefficient = 1 - 2 * area_under_curve

    return gini_coefficient

# before 데이터 결과

In [34]:
before_data=pd.read_csv('before.csv', encoding='cp949')
before_data = before_data.drop(columns=[before_data.columns[0], before_data.columns[10]])
# 예시 데이터프레임을 파이토치 텐서로 변환
new_data_tensor = torch.tensor(before_data.values, dtype=torch.float32)

# 모델에 예시 데이터프레임 입력하여 예측 수행
with torch.no_grad():
    model.eval()  # 모델을 평가 모드로 설정
    predictions = model(new_data_tensor)

# 예측 결과를 데이터프레임으로 만들기
predictions_before = pd.DataFrame(predictions.numpy(), columns=['쇼핑업', '숙박업', '식음료업', '여가서비스업', '운송업', '전체소비'])

predictions_before['전체소비'].std()

3364639.75

In [35]:
# 예시 데이터
income_data = predictions_before['전체소비']

# 개인별 소득 불균형 지수 계산
gini_index = gini_coefficient(income_data)
print("Gini 불균형 지수:", gini_index)

Gini 불균형 지수: 0.03529155254364014


# after 데이터 결과 1: 이동인구 2000만

In [37]:
after=pd.read_csv('after2000.csv', encoding='cp949')
after = after.drop(columns=[after.columns[0], after.columns[1], after.columns[11]])

after

,기타관광,레저스포츠,문화관광,쇼핑,숙박,역사관광,음식,자연관광,체험관광
0,4681899.0,2.138671e+05,27999477.0,29437557.0,28349901.0,2.165957e+06,9.853565e+07,0.000000,7660.0
1,86946.0,2.179701e+05,5863096.0,9500638.0,2338230.0,8.700055e+05,2.536935e+07,0.000000,17301.0
2,419085.0,5.929650e+05,2505485.0,6420236.0,3277662.0,3.538251e+06,1.182914e+07,154548.000000,564.0
3,25319180.0,7.920651e+05,5323442.0,16538216.0,4454421.0,2.923165e+05,1.592225e+07,0.000000,12375.0
4,20944.0,1.169780e+05,4175451.0,16565924.0,4156550.0,6.988030e+05,2.579309e+07,502648.000000,3576.0
...,...,...,...,...,...,...,...,...,...
245,1850280.0,9.559611e+05,4581285.0,2613557.0,2309272.0,1.016113e+06,1.293026e+07,494237.190476,92338.0
246,223980.0,1.240947e+06,1457317.0,3362332.0,423278.0,2.588155e+05,1.875895e+07,0.000000,433519.0
247,4895284.0,1.277867e+06,2365508.0,1177061.0,2141527.0,3.111045e+05,1.073850e+07,129837.190476,41590.0
248,8888731.0,1.107565e+06,1334667.0,5943814.0,2462568.0,2.823052e+04,1.166779e+07,0.000000,14205.0


In [38]:
# 예시 데이터프레임을 파이토치 텐서로 변환
new_data_tensor = torch.tensor(after.values, dtype=torch.float32)

# 모델에 예시 데이터프레임 입력하여 예측 수행
with torch.no_grad():
    model.eval()  # 모델을 평가 모드로 설정
    predictions = model(new_data_tensor)

# 예측 결과를 데이터프레임으로 만들기
predictions_after = pd.DataFrame(predictions.numpy(), columns=['쇼핑업', '숙박업', '식음료업', '여가서비스업', '운송업', '전체소비'])

predictions_after['전체소비'].std()

3363837.5

In [39]:
income_data = predictions_after['전체소비']
gini_index = gini_coefficient(income_data)
print("Gini 불균형 지수:", gini_index)

Gini 불균형 지수: 0.03494119644165039


# after 데이터 결과 2: 이동인구 347만 (10%)

In [40]:
after=pd.read_csv('after347.csv', encoding='cp949')
after = after.drop(columns=[after.columns[0], after.columns[1], after.columns[11]])

# 예시 데이터프레임을 파이토치 텐서로 변환
new_data_tensor = torch.tensor(after.values, dtype=torch.float32)

# 모델에 예시 데이터프레임 입력하여 예측 수행
with torch.no_grad():
    model.eval()  # 모델을 평가 모드로 설정
    predictions = model(new_data_tensor)

# 예측 결과를 데이터프레임으로 만들기
predictions_after = pd.DataFrame(predictions.numpy(), columns=['쇼핑업', '숙박업', '식음료업', '여가서비스업', '운송업', '전체소비'])

income_data = predictions_after['전체소비']
gini_index = gini_coefficient(income_data)
print("Gini 불균형 지수:", gini_index)

Gini 불균형 지수: 0.03524911403656006


# after 데이터 결과 3: 이동인구 696만 (20%)

In [41]:
after=pd.read_csv('after696.csv', encoding='cp949')
after = after.drop(columns=[after.columns[0], after.columns[1], after.columns[11]])

# 예시 데이터프레임을 파이토치 텐서로 변환
new_data_tensor = torch.tensor(after.values, dtype=torch.float32)

# 모델에 예시 데이터프레임 입력하여 예측 수행
with torch.no_grad():
    model.eval()  # 모델을 평가 모드로 설정
    predictions = model(new_data_tensor)

# 예측 결과를 데이터프레임으로 만들기
predictions_after = pd.DataFrame(predictions.numpy(), columns=['쇼핑업', '숙박업', '식음료업', '여가서비스업', '운송업', '전체소비'])

income_data = predictions_after['전체소비']
gini_index = gini_coefficient(income_data)
print("Gini 불균형 지수:", gini_index)

Gini 불균형 지수: 0.03512352705001831


# after 데이터 결과 4: 이동인구 1044만 (30%)

In [42]:
after=pd.read_csv('after1044.csv', encoding='cp949')
after = after.drop(columns=[after.columns[0], after.columns[1], after.columns[11]])

# 예시 데이터프레임을 파이토치 텐서로 변환
new_data_tensor = torch.tensor(after.values, dtype=torch.float32)

# 모델에 예시 데이터프레임 입력하여 예측 수행
with torch.no_grad():
    model.eval()  # 모델을 평가 모드로 설정
    predictions = model(new_data_tensor)

# 예측 결과를 데이터프레임으로 만들기
predictions_after = pd.DataFrame(predictions.numpy(), columns=['쇼핑업', '숙박업', '식음료업', '여가서비스업', '운송업', '전체소비'])

income_data = predictions_after['전체소비']
gini_index = gini_coefficient(income_data)
print("Gini 불균형 지수:", gini_index)

Gini 불균형 지수: 0.03504371643066406
